In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
cols = [ 'expiration_id', 'protocol', 'src_port', 'dst_port',
       'ip_version',   'bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps',
       'src2dst_min_ps', 'src2dst_mean_ps', 'src2dst_stddev_ps',
       'src2dst_max_ps', 'dst2src_min_ps', 'dst2src_mean_ps',
       'dst2src_stddev_ps', 'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms',
       'src2dst_max_piat_ms', 'dst2src_min_piat_ms',
       'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'bidirectional_syn_packets', 'bidirectional_ack_packets',
       'bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets', 'src2dst_ack_packets',
       'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets',
       'dst2src_fin_packets','application_name',
       'application_category_name', 'application_is_guessed',
       'application_confidence', 'content_type', 'udps.num_pkts_up_to_128_bytes',
       'udps.num_pkts_128_to_256_bytes', 'udps.num_pkts_256_to_512_bytes',
       'udps.num_pkts_512_to_1024_bytes',
       'udps.num_pkts_1024_to_1514_bytes', 'udps.min_ttl', 'udps.max_ttl',
       'udps.min_ip_pkt_len', 'udps.max_ip_pkt_len', 'udps.src2dst_flags',
       'udps.dst2src_flags', 'udps.tcp_flags', 'udps.tcp_win_max_in',
       'udps.tcp_win_max_out', 'udps.icmp_type', 'udps.icmp_v4_type',
       'udps.dns_query_id', 'udps.dns_query_type', 'udps.dns_ttl_answer',
       'udps.ftp_command_ret_code', 'udps.retransmitted_in_packets',
       'udps.retransmitted_out_packets', 'udps.retransmitted_in_bytes',
       'udps.retransmitted_out_bytes', 'udps.src_to_dst_second_bytes',
       'udps.dst_to_src_second_bytes', 'udps.src_to_dst_avg_throughput',
       'udps.dst_to_src_avg_throughput', 'udps.src_to_dst_second_bytes2',
       'udps.dst_to_src_second_bytes2', 'udps.src_to_dst_avg_throughput2',
       'udps.dst_to_src_avg_throughput2', 'udps.tcp_init_ms',
       'udps.tcp_synack_ack_ms', 'udps.tcp_half_closed_time_ms',
       'udps.num_pkts_after_termination',
       'udps.src2dst_first_packet_payload_len',
       'udps.dst2src_first_packet_payload_len',
       'udps.bidirectional_transport_bytes',
       'udps.bidirectional_payload_bytes', 'udps.src2dst_transport_bytes',
       'udps.src2dst_payload_bytes', 'udps.dst2src_transport_bytes',
       'udps.dst2src_payload_bytes',
       'udps.src2dst_most_freq_payload_ratio',
       'udps.src2dst_most_freq_payload_len',
       'udps.dst2src_most_freq_payload_ratio',
       'udps.dst2src_most_freq_payload_len',
       'udps.bidirectional_mean_packet_relative_times',
       'udps.bidirectional_stddev_packet_relative_times',
       'udps.bidirectional_variance_packet_relative_times',
       'udps.bidirectional_coeff_of_var_packet_relative_times',
       'udps.bidirectional_skew_from_median_packet_relative_times',
       'udps.src2dst_mean_packet_relative_times',
       'udps.src2dst_stddev_packet_relative_times',
       'udps.src2dst_variance_packet_relative_times',
       'udps.src2dst_coeff_of_var_packet_relative_times',
       'udps.src2dst_skew_from_median_packet_relative_times',
       'udps.dst2src_mean_packet_relative_times',
       'udps.dst2src_stddev_packet_relative_times',
       'udps.dst2src_variance_packet_relative_times',
       'udps.dst2src_coeff_of_var_packet_relative_times',
       'udps.dst2src_skew_from_median_packet_relative_times',
       'udps.min_req_res_time_diff', 'udps.max_req_res_time_diff',
       'udps.mean_req_res_time_diff', 'udps.stddev_req_res_time_diff',
       'udps.variance_req_res_time_diff',
       'udps.coeff_of_var_req_res_time_diff',
       'udps.skew_from_median_req_res_time_diff',
       'udps.src2dst_small_packet_payload_packets',
       'udps.src2dst_small_packet_payload_ratio',
       'udps.dst2src_small_packet_payload_packets',
       'udps.dst2src_small_packet_payload_ratio',
       'udps.sent_recv_packet_ratio',
       'udps.bidirectional_ps_first_quartile',
       'udps.bidirectional_ps_second_quartile',
       'udps.bidirectional_ps_third_quartile',
       'udps.bidirectional_ps_median_absoulte_deviation',
       'udps.bidirectional_ps_skewness', 'udps.bidirectional_ps_kurtosis',
       'udps.bidirectional_piat_first_quartile',
       'udps.bidirectional_piat_second_quartile',
       'udps.bidirectional_piat_third_quartile',
       'udps.bidirectional_piat_median_absoulte_deviation',
       'udps.bidirectional_piat_skewness',
       'udps.bidirectional_piat_kurtosis',
       'udps.median_req_res_time_diff', 'Attack']

In [3]:
cols = [ 'expiration_id', 'protocol',
       'ip_version',   'bidirectional_duration_ms', 'bidirectional_packets',
       'bidirectional_bytes', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'bidirectional_min_ps', 'bidirectional_mean_ps',
       'bidirectional_stddev_ps', 'bidirectional_max_ps',
       'src2dst_min_ps', 'src2dst_mean_ps', 'src2dst_stddev_ps',
       'src2dst_max_ps', 'dst2src_min_ps', 'dst2src_mean_ps',
       'dst2src_stddev_ps', 'dst2src_max_ps', 'bidirectional_min_piat_ms',
       'bidirectional_mean_piat_ms', 'bidirectional_stddev_piat_ms',
       'bidirectional_max_piat_ms', 'src2dst_min_piat_ms',
       'src2dst_mean_piat_ms', 'src2dst_stddev_piat_ms',
       'src2dst_max_piat_ms', 'dst2src_min_piat_ms',
       'dst2src_mean_piat_ms', 'dst2src_stddev_piat_ms',
       'dst2src_max_piat_ms', 'bidirectional_syn_packets', 'bidirectional_ack_packets',
       'bidirectional_psh_packets', 'bidirectional_rst_packets',
       'bidirectional_fin_packets', 'src2dst_syn_packets', 'src2dst_ack_packets',
       'src2dst_psh_packets', 'src2dst_rst_packets',
       'src2dst_fin_packets', 'dst2src_syn_packets', 'dst2src_ack_packets',
       'dst2src_psh_packets', 'dst2src_rst_packets',
       'dst2src_fin_packets','application_name',
       'application_category_name', 'application_is_guessed',
       'application_confidence', 'content_type', 'Attack']

In [4]:
def port_feature(port):
    if port < 1024:
        return 1
    elif port < 49152 and port >= 1024:
        return 2
    else:
        return 3        

In [5]:
def encode(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    les = {}
    for each in cols:
        le_col = LabelEncoder()
        df[each] = le_col.fit_transform(df[each])
        les[each] = le_col
       
    return df, les

In [6]:
def normalize_content_type(x):
    if str(x).isspace():
        return "unkown/unkown"
    elif "/" not in str(x):
        return str(x)+"/unkown"
    else:
        return x

In [7]:
def save_predictions(predictions, targets, timeout, save_path= "results/RF"):
    name_file_pred = 'predictions_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    name_file_y = 'targets_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"

    pickle.dump(predictions, open(os.path.join(save_path, name_file_pred), 'wb') )
    pickle.dump(targets, open(os.path.join(save_path, name_file_y), 'wb') )
    
def load_predictions(timeout, save_path= "results/RF"):
    name_file_pred = 'predictions_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    name_file_y = 'targets_idle_' + str(timeout[0]) + "_active_" + str(timeout[1]) + ".p"
    
    predictions =  pickle.load(open(os.path.join(save_path, name_file_pred), 'rb') )
    targets =  pickle.load(open(os.path.join(save_path, name_file_y), 'rb') )
    return predictions, targets

In [8]:
timeouts = [(0.5,2), (1, 2), (2,2), (0.5,3), (1,3), (2, 3), (3,3), (0.5,4), (1, 4), (2,4), (3,4), (4,4), (0.5,5), (1,5), (2,5), (3,5), (4,5), (5,5), (0.5, 30), (1, 30), (2,30), (3,30), (4,30), (5,30), (10, 30), (0.5, 60), (1, 60), (2,60), (3,60), (4,60), (5,60), (10, 60)]

In [11]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/basicFeatures/RF/RF_cupid_nfstream_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

# Training

In [12]:
best_f1 = 0
worst_f1 = 1


best_mean, worst_mean, best_std, worst_std = None, None, None, None
save=True



for timeout in timeouts:
    print("Processing timeout : ", timeout)
    idle, active = timeout
    out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/timeouts-IDS/NFStream/extractions/new_idle_{idle}min_active_{active}min/CUPID'
    df = pd.read_csv(out_dir+"/CUPID.csv")
    df = df[~df.Attack.isin(['Nslookup', 'Dnstracer', 'Dig'])]

    df = df.sort_values(by=['bidirectional_last_seen_ms']).reset_index(drop=True)
    df_new = df[cols]
    df_new['application_name'] = df_new['application_name'].apply(lambda x: x.split(".")[0])
    df_new['content_type'] = df_new['content_type'].fillna("unkown/unkown")
    df_new['content_type'] = df_new['content_type'].apply(lambda x: normalize_content_type(x))
    df_new['content_sub_type'] = df_new['content_type'].apply(lambda x: x.split("/")[1])
    df_new['content_type'] = df_new['content_type'].apply(lambda x: x.split("/")[0])
    #df_new['src_port'] = df_new['src_port'].apply(lambda x: port_feature(x))
    #df_new['dst_port'] = df_new['dst_port'].apply(lambda x: port_feature(x))
    df_new = df_new.fillna(0)
    categ_cols = ["application_name", "application_category_name", "content_sub_type", "content_type" ]
    df_new, lbl_encoders = encode(df_new,categ_cols)  
    
    X = df_new.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df_new['Attack']
    
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        
        
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = RandomForestClassifier(random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')


        
    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  (0.5, 2)
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.9582758355716237, 'accMean': 0.9747946527989214, 'recMean': 0.966592595684374, 'precMean': 0.9728189716857321}
Std of all scores:  {'f1Std': 0.04678279892612259, 'accStd': 0.032533293350658146, 'recStd': 0.04932183556765455, 'precStd': 0.04083176481513074}
_______________________________________________
Processing timeout :  (1, 2)
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.9527549084208549, 'accMean': 0.9688747527480199, 'recMean': 0.9656396370425966, 'precMean': 0.9642573520110653}
Std of all scores:  {'f1Std': 0.043733943375527155, 'accStd': 0.032151747675201535, 'recStd': 0.04826690852566474, 'precStd': 0.04223048877056102}
_______________________________________________
Processing timeout :  (2, 2)
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean o

In [13]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  (1, 60)
Mean Scores (Best):  {'f1Mean': 0.9591228130622642, 'accMean': 0.9708042140450018, 'recMean': 0.9668185159065044, 'precMean': 0.9737046024519209}
Std Scores (Best): {'f1Std': 0.045845044017884716, 'accStd': 0.03741939387581613, 'recStd': 0.04844578330360875, 'precStd': 0.039456751982676695}


In [14]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  (4, 30)
Mean Scores (Worst):  {'f1Mean': 0.9518024312757227, 'accMean': 0.9627476442410181, 'recMean': 0.9657553290361858, 'precMean': 0.9615523191312697}
Std Scores (Worst): {'f1Std': 0.04503398475104804, 'accStd': 0.03914892255069631, 'recStd': 0.04718407361899637, 'precStd': 0.04658419345521379}


In [15]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/basicFeatures/RF_cupid_nfstream.json', 'w') as f:
    json.dump(results, f, indent=4)

In [17]:
def load_score(timeout):
    with open(f'../checkpoints/ET/ET_unsw_zeek_{timeout}.json', 'r') as f:
        loaded_results = json.load(f)
    return loaded_results


<function TextIOWrapper.close()>

In [ ]:
best_f1 = 0
best_timeout = None
best_prec = None
best_rec = None
best_acc = None

worst_f1 = 1
worst_acc = None
worst_timeout = None
worst_prec = None
worst_rec = None


for timeout in timeouts:
    loaded_results = load_score(timeout)
    
    if loaded_results['f1_score'] > best_f1: 
        best_timeout = loaded_results['Timeout'] 
        best_f1 = loaded_results['f1_score'] 
        best_acc = loaded_results['accuracy'] 
        best_rec=loaded_results['recall'] 
        best_prec=loaded_results['precision'] 
    
    if loaded_results['f1_score'] <= worst_f1: 
        worst_timeout = loaded_results['Timeout'] 
        worst_f1 = loaded_results['f1_score'] 
        worst_acc = loaded_results['accuracy'] 
        worst_rec=loaded_results['recall'] 
        worst_prec=loaded_results['precision'] 
    

In [13]:
print("Best Timeout Combination: ", best_timeout)
print("Best Accuracy: ", best_acc)
print('Best Macro F1-score: :', best_f1)
print('Best Macro Precision: :', best_prec)
print('Best Macro Recall: :', best_rec)

Best Timeout Combination:  (1, 60)
Best Accuracy:  0.9708042140450018
Best Macro F1-score: : 0.9591228130622642
Best Macro Precision: : 0.9737046024519209
Best Macro Recall: : 0.9668185159065044


In [14]:
print("worst Timeout Combination: ", worst_timeout)
print("worst Accuracy: ", worst_acc)
print('worst Macro F1-score: :', worst_f1)
print('worst Macro Precision: :', worst_prec)
print('worst Macro Recall: :', worst_rec)

worst Timeout Combination:  (4, 30)
worst Accuracy:  0.9627476442410181
worst Macro F1-score: : 0.9518024312757227
worst Macro Precision: : 0.9615523191312697
worst Macro Recall: : 0.9657553290361858


In [15]:
import json

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Accuracy': best_acc,
        'F1 Score': best_f1,
        'Precision': best_prec,
        'Recall': best_rec
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Accuracy': worst_acc,
        'F1 Score': worst_f1,
        'Precision': worst_prec,
        'Recall': worst_rec
    },
    
    'Difference': {
        'Accuracy': (best_acc - worst_acc)*100,
        'F1 Score': (best_f1 - worst_f1)*100,
        'Precision': (best_prec - worst_prec)*100,
        'Recall': (best_rec - worst_rec)*100
    }
}



with open('../results/RF_cupid_nfstream.json', 'w') as f:
    json.dump(results, f, indent=4)